## AI Agent Security in Production
### E2B Code Sandbox with Google Gemini
Execute AI-generated Python code in a secure sandbox environment.

## Setup and Imports

In [ ]:
"""E2B Code Sandbox with Google Gemini."""
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import base64
import pandas as pd
import time

root_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root_dir)

from dotenv import load_dotenv
load_dotenv()

from e2b_code_interpreter import Sandbox
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

## Initialize Model and Sandbox

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview")

checkpointer = InMemorySaver()

sbx = Sandbox.create(timeout=1200)
print("Sandbox created")

In [ ]:
execution = sbx.run_code('2+2')
execution

## Helper Functions

In [ ]:
def get_dataset_info(file_path):
    """Get basic dataset info."""
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, nrows=3)
    else:
        df = pd.read_excel(file_path, nrows=3)

    return f"Columns: {list(df.columns)}\nSample data:\n{df.to_string()}"

## Define Tools

In [ ]:
@tool
def upload_file(local_file_name: str):
    """Upload a data file to the E2B sandbox for analysis.
    
    Args:
        local_file_path: Local path to the file (e.g., "./data/IMDB-Movie-Data.csv")
        
    Returns:
        Success message with sandbox_path and dataset_info
    """
    local_file_name = local_file_name.lstrip('/')
    local_file_path = f"./data/{local_file_name}"
    
    if not os.path.exists(local_file_path):
        return f"Error: File not found at {local_file_path}"
        
    with open(local_file_path, "rb") as f:
        sandbox_file = sbx.files.write(f"data/{local_file_name}", f)
    
    dataset_info = get_dataset_info(local_file_path)
    
    return f"File uploaded successfully!\nSandbox path: {sandbox_file.path}\n{dataset_info}"

@tool
def run_python_code(code: str):
    """Execute Python code in E2B sandbox.
    
    Args:
        code: Valid executable Python code. Use print() for output, display(plt.gcf()) for plots.
        
    Returns:
        Execution result
    """
    print('Running code in sandbox....')
    execution = sbx.run_code(code)
    print('Code execution finished!')

    if execution.error:
        return f"Error: {execution.error.name}\nValue: {execution.error.value}"

    os.makedirs('images', exist_ok=True)
    
    output = []
    timestamp = int(time.time())

    output.append(str(execution))
    
    # Save images
    for idx, result in enumerate(execution.results):
        if result.png:
            filename = f'images/{timestamp}_chart-{idx}.png'
            with open(filename, 'wb') as f:
                f.write(base64.b64decode(result.png))
            output.append(f'Chart saved to {filename}')

    # print('Output:', output)
    
    return "\n".join(output) if output else "Code executed successfully"

## Create Agent and Execute Query

In [ ]:
system_prompt = """You are a data analysis assistant. You MUST use the available tools to complete tasks.

AVAILABLE TOOLS:
1. glob_search - Search for files in LOCAL filesystem only (searches ./data directory on your machine)
2. upload_file - Upload files from local to sandbox
3. run_python_code - Execute Python code in sandbox environment

FILE LOCATIONS:
- Local files: Use glob_search to find files in ./data directory
- Sandbox files: After upload, files are stored in '/home/user/data/' directory in code environment
- To check sandbox files: Use run_python_code with 'import os; print(os.listdir("/home/user/data/"))'

WORKFLOW - Follow these steps:
1. Search for data files using glob_search (for LOCAL file discovery only)
2. Upload file using upload_file (transfers from local to sandbox)
3. Generate and execute Python code using run_python_code (all code runs in sandbox where files are in '/home/user/data/')

VISUALIZATION RULES:
- Only create plots if user explicitly asks for: "plot", "chart", "graph", "visualize", "show"
- If plots requested: use matplotlib, add title, axis labels, display(plt.gcf())
- Otherwise: use print() for results

CRITICAL: You MUST call the appropriate tool for each step. Do not just think - ACT by calling tools."""


In [ ]:
from langchain.agents.middleware import FilesystemFileSearchMiddleware

agent = create_agent(
    model=model,
    tools=[upload_file, run_python_code],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[FilesystemFileSearchMiddleware(
        root_path="./data",
        use_ripgrep=True,
        max_file_size_mb=100
    )]
)



In [ ]:
config = {"configurable": {"thread_id": "movie-1"}}

query = "Upload IMDB-Movie-Data.csv and create a line chart showing average ratings over years"

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)

response = result['messages'][-1].text
print(f"\nResponse:\n{response}")

In [ ]:
query = "show me how many files are there? print all files."

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

In [ ]:
config = {"configurable": {"thread_id": "finance-1"}}

query = "Analyze apple 2024 and calculate financial ratios"

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

In [ ]:
query = "analyze google cashflow for 2024 and apple cashflow 2024. compare financial ratios. also show comparison table"

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

In [ ]:
query = """Analyze the 2024 cash flow and financial performance of Google (Alphabet) and Apple.
Compute key financial ratios for both companies, including Gross Profit Margin,
Net Profit Margin, ROA, ROE, Current Ratio, and Debt-to-Equity Ratio.

Create the following visualizations:
1. A grouped bar chart comparing Apple vs Google for each financial ratio
   (x-axis: ratio names, y-axis: ratio values, separate bars for Apple and Google).
2. A comparison table summarizing all calculated ratios side by side.

Add clear titles, axis labels, legends, and use distinct colors for each company.
After generating the plots and table, interpret what the visual comparison reveals
about profitability, efficiency, liquidity, and leverage differences between Apple
and Google in 2024.
"""

config = {"configurable": {"thread_id": "finance-plots-1"}}

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

In [ ]:
query = """How is Alphabet funding its investing cash outflows: operating cash, debt, or cash reserves? Make sure to keep the currency denomination in your final answer
"""

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

In [ ]:
query = """
Using titanic.csv, calculate the survival rate for each passenger class (Pclass) and create a bar chart where 
the x-axis is Pclass (1, 2, 3) and the y-axis is survival rate (percentage of passengers who survived). 
Label the axes clearly and add a title explaining the insight.
"""

config = {"configurable": {"thread_id": "titanic-1"}}

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

In [ ]:
query = """
From titanic.csv, group passengers by Sex and Survived, 
then create a grouped bar chart where the x-axis is Sex (male, female), 
the y-axis is passenger count, and bars are split by survival status (0 = died, 1 = survived). 
Include a legend and interpret the result.
"""

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

In [ ]:
query = """
Using the Fare column from titanic.csv, create a box plot comparing 
ticket fares of survivors and non-survivors, with Survived (0 = did not survive, 1 = survived)
 on the x-axis and Fare on the y-axis. Apply a logarithmic scale to the Fare axis 
 if the values are highly skewed. Use distinct colors for each survival group, 
 add clear axis labels, and set the plot title to “Ticket Fare Distribution by Survival 
 Status on the Titanic”. After generating the plot, briefly explain what it reveals about 
 the relationship between socioeconomic status and survival probability.
"""

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

In [ ]:
from datetime import datetime, timedelta

today = datetime.now()
checkin_date = today
checkout_date = today + timedelta(days=5)

str(today.date()), str(checkin_date.date()), str(checkout_date.date())

In [ ]:
checkout_date.date()